In [1]:
from scrampy import splice
import pandas as pd

Read in Blueprints
------------------

In [19]:
ntf = pd.read_csv('aud/NTF/NotTheFall_sentence_blueprint.csv')
pie = pd.read_csv('aud/pieman/pieman_blueprint.csv')
pie

,end,name,order,start,length
0,26970,pieman,0,14000,12970
1,41620,pieman,1,26970,14650


Make Intact Blueprint
--------------------------------------

In [22]:
# simple intact
intact = pd.concat([ntf, pie])
intact

,end,name,order,start,length
0,8921,NTF,0,0,8921
1,19694,NTF,1,8921,10773
0,26970,pieman,0,14000,12970
1,41620,pieman,1,26970,14650


In [25]:
gaps_intact = splice.insert_gaps(intact)
gaps_intact

,name,start,end,length,order
0,NTF,0,8921,8921,0
1,gap,0,1579,1579,-1
2,NTF,8921,19694,10773,1
3,gap,0,1227,1227,-1
4,pieman,14000,26970,12970,0
5,gap,0,2030,2030,-1
6,pieman,26970,41620,14650,1
7,gap,0,1850,1850,-1


Make Interleaved Blueprint
--------------------------

In [34]:
# loc grabs row using its index value (e.g. 0 or 1)
interl = intact.loc[[0,1]]

In [35]:
splice.insert_gaps(interl)

,name,start,end,length,order
0,NTF,0,8921,8921,0
1,gap,0,1579,1579,-1
2,pieman,14000,26970,12970,0
3,gap,0,2030,2030,-1
4,NTF,8921,19694,10773,1
5,gap,0,1227,1227,-1
6,pieman,26970,41620,14650,1
7,gap,0,1850,1850,-1


Full Example
------------

In [39]:
from scrampy import splice
import pandas as pd
from pydub.audio_segment import AudioSegment
from pydub.playback import play

bp_cat = pd.concat([
    pd.read_csv('aud/NTF/NotTheFall_sentence_blueprint.csv'),
    pd.read_csv('aud/pieman/pieman_blueprint.csv')
    ])

bp_out = splice.insert_gaps(bp_cat.loc[[0,1]])
aud_paths = dict(pieman = 'aud/pieman/pieman.mp3', 
                 NTF = 'aud/NTF/NotTheFall.mp3', 
                 gap = 'aud/silence.mp3')
aud_files = {k: AudioSegment.from_mp3(v) for k,v in aud_paths.items()}

# interleaved audio
audio = splice.aud_from_log(bp_out, **aud_files)

#play(audio)

Using a Simple log of Breakpoints Instead
-----------------------------------------

The code below shows how to format a simple log of splitpoints.
Once formatted, it can be combined with other blueprints as above.

In [14]:
# read in data as 
#   - name of audio 
#   - break points (in clock time)
splits = pd.read_csv('aud/NTF/NotTheFall_sentence_splitpoints.txt')
splits

,name,break
0,NTF,0:00.000
1,NTF,0:08.921
2,NTF,0:19.694


In [15]:
# parse to blueprint
bp = splice.parse_splits(splits)
bp

,name,start,end,length,order
0,NTF,0,8921,8921,0
1,NTF,8921,19694,10773,1
